<a href="https://colab.research.google.com/github/Leucocitokiller/Proyecto-Fina-NLP/blob/main/Proyecto_final_NLP_Redes_Neuronales_Libenson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introducción.**

En este trabajo práctico se aborda el análisis y clasificación de opiniones de usuarios mediante técnicas de Procesamiento de Lenguaje Natural (NLP) y Machine Learning. Se emplean dos conjuntos de datos distintos, provenientes de plataformas reconocidas: Yelp, que contiene reseñas de locales de comida, y Amazon, que incluye comentarios sobre productos.

# **Objetivos.**

El objetivo principal es desarrollar un modelo capaz de identificar automáticamente si un comentario es positivo o negativo, independientemente de la temática o sector al que pertenezca. Para ello, se aplican diferentes herramientas y técnicas propias del NLP, tales como el tokenizado, lemmatización, entre otras, que permiten transformar los textos en formatos adecuados para su análisis computacional.

Posteriormente, se prueba una variedad de modelos de machine learning para evaluar cuál es el más efectivo en la clasificación de sentimientos en ambos datasets. Esto incluye desde modelos clásicos hasta técnicas más avanzadas, buscando generalizar el aprendizaje para que el modelo pueda detectar la polaridad del comentario más allá del contexto específico.

Este enfoque facilita no solo el entendimiento de las opiniones expresadas por los usuarios, sino que también permite desarrollar sistemas automatizados de análisis de sentimientos útiles en distintas aplicaciones comerciales y de investigación.

# **Origen de los datos.**

Los datos pertenecen a una adaptación de los comentarios de yelp y amazon y fueron obtenidos del siguiente link de Github:

https://github.com/luisFernandoCastellanosG/Machine_learning/blob/master/2-Deep_Learning/PLN/Datasets/DataSetOpiniones.zip

Datos del autor:

https://github.com/luisFernandoCastellanosG/Machine_learning/blob/master/2-Deep_Learning/PLN/Datasets/readme.md



# **Desarrollo.**

# **Importación de Librerías.**

In [ ]:
import urllib.request
import numpy as np
import pandas as pd
import os
import time
import sys
#-----librerias para trabajar NLP
!python -m spacy download es_core_news_md
import spacy
import es_core_news_md
#es_core_news_md Medium (modelo mediano):
#Es más pesado y más lento que el sm, pero mucho más preciso. Tiene vectores de palabras, entiende mejor el significado de las palabras.

#-----instalación d librerias para análisis de sentimientos.
!pip install spacy spacy-transformers
!pip install pysentimiento
from pysentimiento import create_analyzer

#----librerias para normalización de textos
import re
from unicodedata import normalize
import unicodedata
from collections import Counter


#----librerias para graficar y wordcloud.
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

#----librerías para trabajar con TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
#----libreria para trabajar con BoW.
from sklearn.feature_extraction.text import CountVectorizer
#----librerias para Machine learning
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score
#----librerias de Redes Neuronales.
# Importamos el Tokenizer para procesar el texto y convertirlo en secuencias numéricas
from tensorflow.keras.preprocessing.text import Tokenizer
# Importamos la función para rellenar las secuencias con ceros y asegurarnos que todas tengan la misma longitud
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
# Importamos el modelo secuencial de Keras, que permite apilar capas de manera lineal
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
!pip install keras-tuner
import keras_tuner as kt
# Importamos las capas necesarias:
# - Embedding: para convertir índices de palabras en vectores densos.
# - SimpleRNN: una capa recurrente que procesa secuencias de datos.
# - Dense: una capa totalmente conectada, utilizada para la salida del modelo.



# **Procesamiento de la Fuente de Datos.**

## Conexión con la fuente de datos.


Se cargan dos dataset desde Github que contienen comentarios sobre celulares (Amanzon) y servicio de restaurantes (Yelp). Ambos dataset se unifican para tener un mayor volumen de datos para analizar.

Los mismos estan compuestos por dos columnas, una con los comentarios de cada usario registrado y otra con el valor asignado a ese comentario.
Si el comentario tiene un valor 1 se lo considera positivo y si tiene valor 2 como negativo.

In [ ]:
# Diccionario con las fuentes y sus URLs
filepath_dict = {
    'yelp': 'https://raw.githubusercontent.com/Leucocitokiller/Proyecto-Fina-NLP/main/yelp_comentarios.csv',
    'amazon': 'https://raw.githubusercontent.com/Leucocitokiller/Proyecto-Fina-NLP/main/amazon_cells_comentarios.csv'

}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['Comentario', 'Valor'], sep=';', encoding='latin-1')
    df['Origen'] = source  # se agrega una nueva columna para saber si los comentarios son de Yelp o Amazon.
    df_list.append(df)

df = pd.concat(df_list)
df.head(1100)

## Normalización de la Fuente de datos.


### Eliminación de signos de puntuación.

In [ ]:
# Definición de función para eliminar los signos de puntuación utilizando re, pero considerando no borrar las vocales con acento.

def remove_punctuation(text):
    # Normaliza el texto a NFKD para separar letras y sus tildes
    text = unicodedata.normalize('NFKD', text)
    # Elimina los caracteres diacríticos (como las tildes)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    # Elimina todo lo que no sea letras, números o espacios
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


# Aplicar la función a la columna 'review_lower'
df['Comentarios'] = df['Comentario'].apply(remove_punctuation)

In [ ]:
df

### Reducir a minúsculas el texto.

In [ ]:
# Create a new column 'Comentarios_lower' with lowercase values from 'Comentario'
df['Comentarios_lower'] = df['Comentarios'].str.lower()

In [ ]:
df

### Convertir a número la columna Valor para su postprocesamiento.

In [ ]:
# Convertimos la columna rating a valor numérico
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')

In [ ]:
df['Valor']

# **NLP**

## Pre Procesamiento.

### Generación del objeto de SPacy para utilizar en el procesamiento del texto en español.

Antes de aplicar técnicas de análisis de sentimiento, se debe realizar un preprocesamiento del texto que prepare los datos para ser interpretados por modelos de NLP.
En este paso, se lleva a cabo la generación del objeto de spaCy para trabajar con el idioma español, lo cual permite aprovechar herramientas lingüísticas como la tokenización, lematización

In [ ]:
nlp = es_core_news_md.load()

### Convertir texto a minúsculas y Tokenización.

Una de las primeras transformaciones aplicadas es la conversión del texto a minúsculas, lo que ayuda a normalizar las palabras y evitar que el modelo interprete como diferentes aquellas que solo varían en el uso de mayúsculas (por ejemplo, "Bueno" y "bueno"). A continuación, se realiza la tokenización, que consiste en dividir el texto en unidades mínimas llamadas tokens (como palabras, signos de puntuación o números), facilitando el análisis posterior del lenguaje.

In [ ]:
df['Comentarios_tokenizados'] = df['Comentarios_lower'].apply(lambda text: nlp(text))
df[['Comentarios_lower','Comentarios_tokenizados']].head()

### Remoción de StopWords

Durante la eliminación de stopwords, que son palabras vacías o de bajo contenido semántico (como “el”, “la”, “y”, “de”), se identificó que en algunos casos su remoción podía afectar negativamente el sentido original de las frases. Esto es particularmente relevante en reseñas donde expresiones comunes dependen de ciertas palabras funcionales para conservar su significado completo.

Por esta razón, fue necesario generar un listado personalizado de palabras que debían conservarse.

Esto permitió preservar la coherencia y contexto de los comentarios, evitando que el modelo perdiera información clave para la detección del sentimiento.

In [ ]:
# Lista de palabras que NO queremos eliminar (tienen carga emocional)
palabras_sentimiento = {
    # Positivas
    "bueno", "buena","si","buenísimo", "excelentes", "excelente", "genial", "maravilloso", "maravilla", "fantástico", "fabuloso", "increíble",
    "perfecto", "perfecta", "agradable", "satisfecho", "satisfecha", "contento", "contenta", "encantado", "encantada",
    "amable", "simpático", "simpática", "rápido", "rápida", "cómodo", "cómoda", "eficaz", "eficiente", "fácil",
    "recomendable", "ideal", "espectacular", "feliz", "brillante", "cumplió", "cumple", "funciona", "funciona bien",
    "inmejorable", "confiable", "duradero", "cumplidor", "seguro", "preciso", "elegante", "atento", "responsable",
    "acertado", "destacado", "excepcional", "impecable", "sensacional", "útil", "accesible", "económico", "funcional",
    "intuitivo", "conveniente", "hermoso", "linda", "precioso", "excelente calidad", "vale la pena",

    # Negativas
    "malo","no", "mala", "mal", "pésimo", "pésima","nunca", "horrible", "fatal", "insoportable", "lento", "lenta", "incómodo", "incómoda",
    "decepcionante", "decepcionado", "decepcionada", "sucio", "sucia", "caro", "cara", "inútil", "deficiente", "desagradable",
    "complicado", "problemático", "estafa", "engañado", "engañada", "roto", "rota", "desastroso", "error", "errores",
    "retraso", "tardanza", "frágil", "inestable", "poco fiable", "nunca más", "no volveré", "no recomiendo", "no sirve",
    "no funciona", "arruinado", "falló", "fallando", "demora", "pésima atención", "servicio malo", "mala calidad", "molesto",
    "defecto", "problemas", "fallas", "sin sentido", "basura", "pérdida de dinero", "decepción"
}

# Actualizamos spaCy para que NO considere esas palabras como stopwords
for palabra in palabras_sentimiento:
    lex = nlp.vocab[palabra]
    lex.is_stop = False

def parse_and_remove_stopwords(doc):
    """
    Remueve las stopwords de un objeto spaCy Doc.
    """
    # Filtrar stopwords y obtener los tokens como texto
    tokens_filtrados = [token.text for token in doc if not token.is_stop]
    return tokens_filtrados

# Aplicar la función al DataFrame
df['Comentarios_sin_StopWords'] = df['Comentarios_tokenizados'].apply(parse_and_remove_stopwords)

df[['Comentarios_tokenizados','Comentarios_sin_StopWords']].head()

### Lematizado.

Se procede a aplicar la lematización, una técnica que permite reducir cada palabra a su forma base o "lema". Por ejemplo, palabras como “comprando”, “compré” o “comprarían” se transforman en “comprar”. Esto es esencial para evitar la dispersión semántica y lograr que el modelo reconozca distintas variantes de una palabra como una misma entidad.

In [ ]:
def lematizar_sin_stopwords(doc):
    """
    Devuelve una lista de lemas excluyendo las stopwords.

    Parámetro:
    - doc: objeto spaCy Doc

    Retorna:
    - Lista de lemas (str) sin stopwords
    """
    return [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

# Aplicar la función y guardar el resultado en una nueva columna
df['Comentarios_lema'] = df['Comentarios_tokenizados'].apply(lematizar_sin_stopwords)

df[['Comentarios_tokenizados','Comentarios_lema']].head(20)

## Procesamiento

### Conteo de Palabras mas comunes.

Como parte del análisis exploratorio, se realizó un conteo de las palabras más frecuentes dentro de los comentarios.
Esta etapa permite identificar los términos que predominan en el lenguaje utilizado por los usuarios y detectar patrones o temas recurrentes en las opiniones.

Para un análisis más detallado, el conteo se dividió entre los comentarios de Yelp y Amazon, con el fin de comparar el vocabulario característico de cada plataforma. Mientras Yelp tiende a centrarse en experiencias relacionadas con servicios (como restaurantes o locales comerciales), Amazon refleja opiniones sobre productos, lo cual se evidencia en las diferencias léxicas observadas entre ambos conjuntos de datos.

In [ ]:
def graficar_palabras_comunes(df, origen, top_n=10):
    # Filtrar y aplanar los lemas
    lemas = [lema for lemas in df[df['Origen'] == origen]['Comentarios_lema'] for lema in lemas]
    conteo = Counter(lemas).most_common(top_n)

    # Separar palabras y frecuencias
    palabras, frecuencias = zip(*conteo)

    # Crear gráfico
    plt.figure(figsize=(10, 6))
    plt.barh(palabras, frecuencias, color='skyblue')
    plt.xlabel('Frecuencia')
    plt.title(f'Top {top_n} Palabras Más Comunes - {origen.capitalize()}')
    plt.gca().invert_yaxis()  # Poner la palabra más común arriba
    plt.tight_layout()
    plt.show()

# Graficar para Yelp
graficar_palabras_comunes(df, 'yelp')

# Graficar para Amazon
graficar_palabras_comunes(df, 'amazon')

### Conteo de bigramas más comunes.

Además del análisis de palabras individuales, se llevó a cabo un conteo de bigramas (pares de palabras consecutivas) con el objetivo de capturar expresiones más completas y contextuales utilizadas por los usuarios en sus comentarios. A diferencia del análisis unigram (una sola palabra), los bigramas permiten identificar frases frecuentes que pueden tener un valor semántico más claro, como "muy bueno", "no funciona", "excelente producto", entre otros.

In [ ]:
from collections import Counter
from itertools import tee
import matplotlib.pyplot as plt

def generar_bigramas(lista):
    """Devuelve bigramas como tuplas a partir de una lista de palabras"""
    a, b = tee(lista)
    next(b, None)
    return list(zip(a, b))

def graficar_bigramas_comunes(df, origen, top_n=10):
    # Filtrar solo los comentarios del origen y generar bigramas
    bigramas = [
        bigrama
        for lemas in df[df['Origen'] == origen]['Comentarios_lema']
        for bigrama in generar_bigramas(lemas)
    ]

    conteo = Counter(bigramas).most_common(top_n)

    # Convertir tuplas de bigramas a string para graficar
    etiquetas = [' '.join(b) for b, _ in conteo]
    frecuencias = [f for _, f in conteo]

    # Crear gráfico
    plt.figure(figsize=(10, 6))
    plt.barh(etiquetas, frecuencias, color='mediumseagreen')
    plt.xlabel('Frecuencia')
    plt.title(f'Top {top_n} Bigramas Más Comunes - {origen.capitalize()}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# Ejemplo de uso
graficar_bigramas_comunes(df, 'yelp')
graficar_bigramas_comunes(df, 'amazon')


### WordClouds

Para complementar el análisis exploratorio, se generaron nubes de palabras que representan gráficamente la frecuencia de aparición de términos en los comentarios. Este tipo de visualización permite identificar rápidamente las palabras y frases más utilizadas por los usuarios, otorgando una vista intuitiva del contenido predominante en cada dataset.

Se realizaron dos tipos de word clouds:

Una para monogramas, es decir, palabras individuales.

Otra para bigramas, que agrupa las dos palabras consecutivas más frecuentes.

Ambos casos compara a Yelp y Amazon.

#### WordCloud Yelp.

In [ ]:
# Filtrar el DataFrame
df_yelp = df[df['Origen'] == 'yelp']

# Unir todos los lemas en un solo string (comentarios lematizados ya están en listas)
texto_yelp = ' '.join([' '.join(lemas) for lemas in df_yelp['Comentarios_lema']])

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_yelp)

# Mostrar la nube
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Nube de Palabras - Comentarios de Yelp")
plt.show()

#### WordCloud Amazon.

In [ ]:
# Filtrar el DataFrame
df_amazon = df[df['Origen'] == 'amazon']

# Unir todos los lemas en un solo string (comentarios lematizados ya están en listas)
texto_amazon = ' '.join([' '.join(lemas) for lemas in df_amazon['Comentarios_lema']])

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_amazon)

# Mostrar la nube
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Nube de Palabras - Comentarios de Yelp")
plt.show()


#### WordCloud + Bigramas.

In [ ]:
def generar_bigramas_spacy(df, origen, top_n=50):
    """
    Genera bigramas usando spaCy a partir de la columna 'Comentarios_Lema', sin stopwords.
    Luego genera una nube de palabras.
    """
    # Filtrar los comentarios por 'origen' (por ejemplo, 'yelp' o 'amazon')
    comentarios = df[df['Origen'] == origen]['Comentarios_sin_StopWords']

    # Generar bigramas
    bigramas = []
    for comentario in comentarios:
        # Crear un Doc de spaCy a partir de la lista de lemas (de la columna 'Comentarios_Lema')
        doc = nlp(' '.join(comentario))  # Unimos la lista de lemas y lo procesamos con spaCy
        # Extraer bigramas
        for i in range(len(doc) - 1):
            if not doc[i].is_stop and not doc[i+1].is_stop:  # Asegurarse de que no sean stopwords
                bigramas.append((doc[i].lemma_, doc[i+1].lemma_))

    # Contar los bigramas más comunes
    conteo_bigramas = Counter(bigramas).most_common(top_n)

    # Convertir los bigramas a formato texto "palabra1 palabra2"
    bigramas_texto = {' '.join(bigrama): freq for bigrama, freq in conteo_bigramas}

    # Generar la nube de palabras de los bigramas
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(bigramas_texto)

    # Mostrar la nube
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Word Cloud de Bigramas - {origen.capitalize()}")
    plt.show()

# Generar la nube de bigramas para Yelp y Amazon
generar_bigramas_spacy(df, 'yelp')
generar_bigramas_spacy(df, 'amazon')

###🚀 Análisis de sentimiento en español con pysentimiento

Una vez preprocesados los textos, se aplicó un modelo de análisis de sentimiento específicamente entrenado para el idioma español utilizando la librería pysentimiento.

pysentimiento permite detectar si un comentario es positivo, negativo o neutral, lo cual resulta fundamental para evaluar la percepción general de los usuarios sobre un producto o servicio. A diferencia de otros enfoques más simples, este modelo considera la estructura gramatical y el significado global de la oración, lo que mejora notablemente la precisión, especialmente en expresiones ambiguas o sarcásticas.

In [ ]:
# Crear analizador de sentimientos
analyzer = create_analyzer(task="sentiment", lang="es")

# Aplicar a una columna de texto
df['Sentimiento'] = df['Comentario'].apply(lambda x: analyzer.predict(x).output)
# Sentimiento solo guarda lo predicho (POS, NEU o NEG)

df['Probabilidad'] = df['Comentario'].apply(lambda x: analyzer.predict(x).probas)
#Ese diccionario contiene la probabilidad de cada clase: positivo, neutro, negativo Ejemplo: {'POS': 0.84, 'NEU': 0.10, 'NEG': 0.06}.

#### 📊  Gráfico de barras de frecuencia de sentimientos

In [ ]:
sns.countplot(data=df, x='Sentimiento', order=['POS', 'NEU', 'NEG'], palette='pastel')
plt.title('Distribución de Sentimientos totales entre Yelp y Amazon')
plt.xlabel('Sentimiento')
plt.ylabel('Cantidad de Comentarios')
plt.show()


#### Distribución de los sentimientos.

In [ ]:
df['Sentimiento'].value_counts().plot.pie(
    autopct='%1.1f%%',
    startangle=90,
    labels=['Positivo', 'Neutro', 'Negativo'],
    colors=['lightgreen', 'lightblue', 'salmon']
)
plt.title('Distribución porcentual de Sentimientos')
plt.ylabel('')
plt.show()

#### Análisis de Confianza para filtrar comentarios con baja certeza

Luego de aplicar el modelo de análisis de sentimiento con pysentimiento, se incorporó una etapa adicional para evaluar la confianza de las predicciones. Este análisis se basa en las probabilidades asignadas a cada clase (positivo, negativo o neutral), lo que permite identificar qué tan seguro está el modelo respecto a cada clasificación realizada.

Con esta información se implementó un filtro de confianza, excluyendo o marcando aquellos comentarios cuya predicción tiene baja certeza (por ejemplo, menor al 60%). Esta estrategia ayuda a reducir errores en el análisis general, ya que evita tomar decisiones basadas en clasificaciones inciertas o ambiguas.

Además, el análisis de confianza permite estudiar qué tipos de comentarios generan más dudas en el modelo, lo que puede ser útil para mejorar el preprocesamiento, ajustar umbrales, o incluso etiquetar manualmente ciertos casos en futuras iteraciones del modelo.

In [ ]:
# Máxima probabilidad (nivel de certeza del modelo)
df['Confianza'] = df['Probabilidad'].apply(lambda x: max(x.values()))  # En este caso, de la lista {'POS': 0.84, 'NEU': 0.10, 'NEG': 0.06} sólo guarda 0.84 que es el valor mayor

# Filtrar comentarios cuya confianza sea menor a 0.6
comentarios_baja_confianza = df[df['Confianza'] < 0.6]
comentarios_alta_confianza = df[df['Confianza'] >= 0.6]
# Ver los primeros resultados
#comentarios_baja_confianza[['Comentarios', 'Sentimiento', 'Confianza']]

#### Distribución de los comentarios filtrados.

In [ ]:
df[df['Confianza'] >= 0.6]['Sentimiento'].value_counts().plot.pie(
    autopct='%1.1f%%',
    startangle=90,
    labels=['Positivo', 'Neutro', 'Negativo'],
    colors=['lightgreen', 'lightblue', 'salmon']
)
plt.title('Distribución porcentual de Sentimientos con alta confianza')
plt.ylabel('')
plt.show()

#### Distribución de la Confianza de los comentarios.

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data=df, x='Confianza', bins=20, kde=True, color='skyblue')
plt.axvline(0.6, color='red', linestyle='--', label='Umbral 0.6')
plt.title('Distribución de Confianza del Sentimiento')
plt.xlabel('Confianza')
plt.ylabel('Cantidad de Comentarios')
plt.legend()
plt.show()

In [ ]:
comentarios_baja_confianza[['Comentario', 'Sentimiento', 'Confianza', 'Probabilidad']].sort_values(by='Confianza').head(20)


# **Pruebas de Modelos de Machine Learning.**

En este punto, se realizará un análisis de texto con el objetivo de predecir una variable numérica que nos indica positvo (para 1) y negativo (para 0), a partir de opiniones o reseñas escritas por usuarios.


Para convertir los textos en datos numéricos que puedan ser procesados por modelos de machine learning, seutilizaran dos técnicas de representación de texto:TF-IDF (Term Frequency-Inverse Document Frequency) y Bag of Words (BoW).


TF-IDF pondera la frecuencia de las palabras en cada documento ajustándola según su frecuencia inversa en todo el corpus, dando mayor peso a términos distintivos y reduciendo la influencia de palabras comunes.

Bag of Words representa cada documento como un vector que indica la frecuencia de cada palabra, sin considerar el orden ni la relevancia contextual.

Estos vectores van a ser utilizados como entrada para un modelo de regresión logística, que permitirá predecir la variable objetivo asociada a cada texto, en este caso si los comentarios son positivos o negativos.

Adicionalmente, se incorporará un modelo de deep learning utilizando la biblioteca Keras, que aprvechando redes neuronales para capturar patrones más complejos en los textos, incluyendo relaciones contextuales y secuenciales entre palabras que no pueden ser detectadas por las representaciones tradicionales.

Se entrenarán y evaluarán los tres modelos —regresión lineal con Bag of Words, regresión lineal con TF-IDF y red neuronal profunda con Keras— para comparar su desempeño predictivo.

La evaluación incluirá métricas adecuadas para regresión y análisis de generalización, con el fin de identificar cuál enfoque es más efectivo para este problema específico.

Este análisis permitirá no solo comparar técnicas clásicas y modernas de procesamiento de texto, sino también obtener insights sobre la relevancia y el impacto de las palabras y estructuras en la predicción, mejorando la comprensión del comportamiento del modelo y la calidad de las predicciones.

### División de datos de entrenamiento y prueba.

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df['Comentario'], df['Valor'], test_size=0.2, random_state=42
)

## Regresión Logística.

### Utilizando TF-IFD.

TF-IDF (Term Frequency - Inverse Document Frequency) es una técnica de procesamiento de texto utilizada para evaluar la importancia de una palabra dentro de un conjunto de documentos. Se basa en dos conceptos:

TF (Frecuencia de Término): Mide cuántas veces aparece un término en un documento específico, comparado con el número total de términos en ese documento. Esto ayuda a capturar cuán relevante es una palabra dentro de un documento en particular.

IDF (Frecuencia Inversa de Documentos): Mide la importancia de una palabra dentro de un conjunto de documentos. Si una palabra aparece en muchos documentos, tiene menos valor. La fórmula es:

Esto ayuda a reducir el peso de las palabras que aparecen frecuentemente en todos los documentos (como "el", "y", "de"), ya que no agregan mucha información.

Así, la importancia de un término en un documento depende tanto de su frecuencia en ese documento como de cuán común es en todo el conjunto de documentos.

#### Cálculo de TF-IDF con TfidVetorizer y analisis de n-gramas.

In [ ]:
 # TFIDF espera trabajar con strings y  no listas, por lo que se procede a crear una nueva columna con los datos tokenizados en formato str.
df['Comentarios_sin_StopWords_str'] = df['Comentarios_sin_StopWords'].apply(lambda x: ' '.join(x))

# Crear el vectorizador
tfidfvectorizer = TfidfVectorizer(ngram_range=(1,5))
#inlcuyo bigramas y trigramas para que le de contexto a los comentarios. Esto me permite ver un "No conforme" y no solamente le "No" y el "Conforme" por separado.

# Ajustar y transformar
tfidf_matrix = tfidfvectorizer.fit_transform(df['Comentarios_sin_StopWords_str'])

# Obtener los términos
features = tfidfvectorizer.get_feature_names_out()

# Convertir la matriz a DataFrame
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=features)

# Sumar TF-IDF por columna
tfidf_scores = df_tfidf.sum().sort_values(ascending=False)

# Mostrar top 10
print("🔝 Top 10 n-gramas por score TF-IDF:")
print(tfidf_scores.head(10).round(3))




In [ ]:
# Crear un DataFrame auxiliar con el tipo de n-grama
df_scores = pd.DataFrame({
    'ngram': tfidf_scores.index,
    'score': tfidf_scores.values,
    'tipo': tfidf_scores.index.to_series().apply(lambda x: f'{len(x.split())}-grama')
})

# Ver los 5 más importantes por tipo
top_n = 5
for tipo in ['1-grama', '2-grama', '3-grama']:
    print(f"\n🔝 Top {top_n} {tipo}s:")
    print(df_scores[df_scores['tipo'] == tipo].head(top_n).to_string(index=False))

El análisis de los n-gramas (1, 2 y 3 palabras) revela una fuerte tendencia negativa en los comentarios analizados.

Esto se evidencia principalmente por:

La presencia dominante de la palabra "no" como unigram (1-grama) más relevante, indicando una alta frecuencia de negaciones.

Los bigramas y trigramas refuerzan esta tendencia negativa, con frases como "no volveremos", "no volvere", "no funciona", "no recomendaria", "no vale pena" y "no compre producto", todas las cuales reflejan insatisfacción o malas experiencias.

Aun así, hay menciones positivas como "buena comida servicio", pero estas son menos frecuentes o tienen menor peso que las negativas.

#### Ajuste de datos de Entrenamiento.

Ajustamos el vectorizador TF-IDF con los datos de entrenamiento y test  transformando esos datos en una matriz numérica.

In [ ]:
# Ajustar y transformar los datos de entrenamiento
X_train_tfidf = tfidfvectorizer.fit_transform(X_train)
# Transformar los datos de prueba
X_test_tfidf = tfidfvectorizer.transform(X_test)

#### Generación y prueba de modelo Regresión Logística.

In [ ]:
# Crear un modelo de regresión logística
# Abajo tenés un código con los parámetros expresados de forma que puedas ir modificandolos
model_log_reg = LogisticRegression() #Instanciamos el modelo

# Entrenar el modelo
model_log_reg.fit(X_train_tfidf, y_train) # Fiteamos, es decir, el modelo aprende a partir de los datos de entrenamiento

# Hacer predicciones en el conjunto de prueba
y_pred_log_reg = model_log_reg.predict(X_test_tfidf) # Predecir

#### Evaluación del Modelo.

Matriz de Confusión.

In [ ]:
# 1. Matriz de confusión

# La Matriz de Confusión es útil para Muestra los aciertos y errores del modelo organizados por clase.

cm = confusion_matrix(y_test, y_pred_log_reg)
labels = ['Negativo', 'Positivo']

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusión')
plt.show()

📊 Interpretación de los resultados:
TP (Verdaderos Positivos) = 148: Casos positivos correctamente clasificados como positivos.

TN (Verdaderos Negativos) = 181: Casos negativos correctamente clasificados como negativos.

FP (Falsos Positivos) = 28: Casos negativos mal clasificados como positivos.

FN (Falsos Negativos) = 33: Casos positivos mal clasificados como negativos.

La matriz de confusión muestra un buen desempeño del modelo:
- 181 verdaderos negativos y 148 verdaderos positivos indican una buena capacidad para clasificar correctamente ambas clases.
- Sin embargo, hay 28 falsos positivos y 33 falsos negativos, lo que sugiere que el modelo comete algunos errores, especialmente en la identificación de la clase positiva.
- Estos errores podrían ser relevantes dependiendo del contexto del problema (por ejemplo, si detectar positivos es crítico).

Curva ROC AUC.

El ROC AUC (Receiver Operating Characteristic - Area Under Curve) es una métrica que mide la capacidad del modelo para distinguir entre clases (positiva y negativa), evaluando todas las combinaciones posibles de umbrales de clasificación.

ROC: Es una curva que grafica la tasa de verdaderos positivos (TPR) contra la tasa de falsos positivos (FPR) a distintos umbrales.

AUC (Area Under Curve): Es el área bajo esa curva, y su valor va de 0 a 1:

1.0 = modelo perfecto.

0.5 = modelo sin capacidad de clasificación (como adivinar).

< 0.5 = peor que adivinar (clasifica al revés).

In [ ]:
# 2. Curva ROC
fpr, tpr, _ = roc_curve(y_test, model_log_reg.decision_function(X_test_tfidf))
roc_auc = roc_auc_score(y_test, model_log_reg.decision_function(X_test_tfidf))

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Área bajo la curva ROC AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

📈 Interpretación de tu resultado (AUC = 0.89):
El valor 0.89 indica que el modelo tiene una alta capacidad para distinguir entre clases.

En promedio, hay un 89% de probabilidad de que el modelo asigne un mayor score a una instancia positiva que a una negativa.

Este resultado sugiere que el modelo está haciendo un buen trabajo, incluso si aún hay algunos falsos positivos o falsos negativos.

Métricas de Predicción.

- Accuracy:Para medir qué tan bien predice el modelo en datos nuevos (exactitud).
Accuracy mide el porcentaje total de predicciones correctas sobre el total de casos.
- Precision: Para medir el costo de un falso positivo es alto (por ejemplo, recomendar una película mala como buena).
Precision mide qué proporción de las predicciones positivas hechas por el modelo son realmente positivas.
- Recall: Para medir cuántos de los casos positivos reales fueron capturados por el modelo.
- f1 Score: Para medir el promedio armónico entre precisión y recall. Un buen balance si ambas cosas son importantes.

In [ ]:
# 3. Métricas
# Accuracy:Para medir qué tan bien predice el modelo en datos nuevos (exactitud).
# Accuracy mide el porcentaje total de predicciones correctas sobre el total de casos.
accuracy = accuracy_score(y_test, y_pred_log_reg)
# Precision: Para medir el costo de un falso positivo es alto (por ejemplo, recomendar una película mala como buena).
# Precision mide qué proporción de las predicciones positivas hechas por el modelo son realmente positivas.
precision = precision_score(y_test, y_pred_log_reg)
# Recall: Para medir cuántos de los casos positivos reales fueron capturados por el modelo.
recall = recall_score(y_test, y_pred_log_reg)
#
f1 = f1_score(y_test, y_pred_log_reg)

print("Métricas de desempeño del modelo:")
print(f"Accuracy : {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall   : {recall:.2f}")
print(f"F1 Score : {f1:.2f}")

Resultados:

- Accuracy = 0.82
El 82% de todas las predicciones (positivas y negativas) fueron correctas. Es una medida general del rendimiento.
Sin embargo, puede ser engañosa si las clases están desbalanceadas.

- Precision = 0.80
De todas las predicciones positivas que hizo el modelo, el 80% fueron realmente positivas.
Es importante si queremos minimizar falsos positivos (por ejemplo, evitar alarmas innecesarias).

- Recall = 0.82
El modelo identificó correctamente el 82% de todos los casos realmente positivos.
Es importante si queremos minimizar falsos negativos (por ejemplo, no dejar pasar casos positivos importantes).

- F1 Score = 0.81
Es el promedio armónico entre precision y recall. Resume el equilibrio entre ambos.
 Un F1 de 0.81 indica un buen balance entre identificar positivos y no equivocarse al predecirlos.


*Validación Cruzada.*

La validación cruzada es una técnica para evaluar la capacidad de generalización de un modelo. Consiste en:

Dividir los datos en k partes (folds).

Entrenar el modelo con k-1 partes y validar con la parte restante.

Repetir esto k veces, cambiando el fold de validación en cada iteración.

Calcular el promedio de las métricas obtenidas en cada iteración.

Esto reduce el riesgo de que el modelo esté sobreajustado (overfitting) a una única partición de los datos.

In [ ]:
# Pipeline que junta vectorizador y modelo
pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000),
    LogisticRegression()
)

# Validación cruzada con 5 particiones (k-fold = 5)
scores = cross_val_score(pipeline, df['Comentario'], df['Valor'], cv=5, scoring='accuracy')

# Resultados
print(f"Precisión media con validación cruzada: {scores.mean():.3f}")
print(f"Desviación estándar: {scores.std():.3f}")

Resultados de la validación cruzada:

La precisión media del modelo es 0.807, lo que indica que, en promedio, el modelo acierta con un 80.7% de efectividad
en los distintos subconjuntos del conjunto de datos evaluados.

La desviación estándar es 0.018, lo que significa que el desempeño del modelo es bastante consistente entre los diferentes folds.
Es decir, no hay una gran variación en la precisión dependiendo del conjunto de entrenamiento/validación utilizado.
Estos resultados sugieren que el modelo tiene un buen rendimiento general y una buena capacidad de generalización.

Visualización de palabras asociadas a reseñas positivas y negativas.

Esta visualización permite identificar las palabras más frecuentes o relevantes en cada grupo de reseñas, separando aquellas asociadas con opiniones positivas de las relacionadas con opiniones negativas.

Al analizar estas palabras clave, podemos entender mejor qué aspectos del producto o servicio generan satisfacción o insatisfacción en los usuarios.

Este tipo de análisis ayuda a extraer insights cualitativos que complementan las métricas cuantitativas, y es muy útil para mejorar la experiencia del cliente y orientar acciones específicas de mejora.

In [ ]:
# Obtenemos las palabras del vocabulario
palabras = tfidfvectorizer.get_feature_names_out()

# Coeficientes del modelo (uno por palabra)
coeficientes = model_log_reg.coef_[0]

# Creamos un DataFrame para visualizarlo
df_coef = pd.DataFrame({'palabra': palabras, 'coeficiente': coeficientes})

# Ordenamos por importancia
df_coef = df_coef.sort_values(by='coeficiente', ascending=False)

# En la primera columna veremos el número "índice" de cada palabra según el órden en que fueron procesadas en el modelo.

# Mostramos las 10 palabras más asociadas a valoración positiva y negativa
print("🔼 Palabras más asociadas a reseñas positivas:")
print(df_coef.head(10))

print("\n🔽 Palabras más asociadas a reseñas negativas:")
print(df_coef.tail(10))

Las palabras con coeficientes positivos más altos, como "gran", "buen", "excelente", "funciona" y "increíble", están fuertemente asociadas con reseñas positivas, reflejando satisfacción, calidad y buen desempeño del producto o servicio.

En contraste, las palabras con coeficientes negativos más fuertes, como "no", "mala", "decepcionado", "horrible" y "terrible", se asocian claramente con reseñas negativas, indicando insatisfacción, problemas y decepción por parte de los usuarios.

Esto muestra que el modelo ha identificado correctamente los términos que expresan opiniones positivas y negativas, lo que facilita la interpretación y el análisis cualitativo del sentimiento en los textos.

Graficar las 10 palabras más positivas y más negativas

In [ ]:
# Colores pastel suaves (más apagados)
color_positivas = '#388e3c'  # verde claro apagado
color_negativas = '#e65100'  # rojo claro apagado

# Top 10 positivas y negativas
top_positivas = df_coef.head(10)
top_negativas = df_coef.tail(10).sort_values(by='coeficiente')

# Crear la figura y los ejes
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de palabras positivas
bars1 = ax[0].barh(top_positivas['palabra'], top_positivas['coeficiente'], color=color_positivas)
ax[0].set_title('🔼 Palabras asociadas a reseñas positivas', fontsize=14)
ax[0].invert_yaxis()
ax[0].set_xlabel('Coeficiente', fontsize=12)

# Agregar valores al final de las barras (positivas)
for bar in bars1:
    width = bar.get_width()
    ax[0].text(width + 0.01, bar.get_y() + bar.get_height() / 2,
               f'{width:.2f}', va='center', fontsize=10)

# Gráfico de palabras negativas
bars2 = ax[1].barh(top_negativas['palabra'], top_negativas['coeficiente'], color=color_negativas)
ax[1].set_title('🔽 Palabras asociadas a reseñas negativas', fontsize=14)
ax[1].invert_yaxis()
ax[1].set_xlabel('Coeficiente', fontsize=12)

# Agregar valores al final de las barras (negativas)
for bar in bars2:
    width = bar.get_width()
    ax[1].text(width - 0.01, bar.get_y() + bar.get_height() / 2,
               f'{width:.2f}', va='center', ha='right', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Filtrar bigramas y trigramas
# Cambiar 'ngrama' a 'palabra' para acceder a la columna correcta
df_bi_tri = df_coef[df_coef['palabra'].str.count(' ') >= 2]

top_pos_bi_tri = df_bi_tri.sort_values('coeficiente', ascending=False).head(10)
top_neg_bi_tri = df_bi_tri.sort_values('coeficiente', ascending=True).head(10)

color_positivas = '#388e3c'  # verde oscuro
color_negativas = '#e65100'  # naranja oscuro

fig, axes = plt.subplots(1, 2, figsize=(16,6))

axes[0].barh(top_pos_bi_tri['palabra'], top_pos_bi_tri['coeficiente'], color=color_positivas) # Cambiar 'ngrama' a 'palabra'
axes[0].invert_yaxis()
axes[0].set_title('🔼 Bigrams y Trigrams positivos')
axes[0].set_xlabel('Coeficiente')

axes[1].barh(top_neg_bi_tri['palabra'], top_neg_bi_tri['coeficiente'], color=color_negativas) # Cambiar 'ngrama' a 'palabra'
axes[1].invert_yaxis()
axes[1].set_title('🔽 Bigrams y Trigrams negativos')
axes[1].set_xlabel('Coeficiente')

plt.tight_layout()
plt.show()


🧠 ¿Qué muestran los gráficos?
Las palabras con coeficientes positivos son las que más contribuyen a que el modelo prediga una reseña positiva.

Las palabras con coeficientes negativos son las que más empujan al modelo hacia una predicción negativa.

Prueba del modelo.

Se genera un código para probar distintas frases con comentarios genericos con el motivo de evaluar la funcionalidad del modelo de predicción.

In [ ]:
nueva_reseña = "no lo recominedo"  # Reemplaza con la reseña que deseas probar
nueva_reseña_tfidf = tfidfvectorizer.transform([nueva_reseña])
prediccion = model_log_reg.predict(nueva_reseña_tfidf)
# Obtener la probabilidad de la predicción
probabilidadpositiva = model_log_reg.predict_proba(nueva_reseña_tfidf)

# Obtener la probabilidad en la clase predicha (0 o 1)
probabilidad = probabilidadpositiva[0][1]  # Probabilidad de la clase "positivo"

print(f"Se predice que la crítica es de caracter {prediccion[0]}")
print(f" con una probabilidad de que sea positiva de {probabilidad:.2f}")

### Utilizando Bag of Words.

BoW convierte un conjunto de documentos en una matriz de ocurrencias de palabras. A diferencia de TF-IDF, que pondera las palabras según su frecuencia e importancia en relación con todo el corpus, BoW solo cuenta cuántas veces aparece una palabra en un documento sin considerar la frecuencia global de la palabra.

#### Ajuste de datos de entrenamiento para BoW.

In [ ]:
# Instanciamos el vectorizador BoW
vectorizer_bow = CountVectorizer()

# Aplicamos el vectorizador a los comentarios lematizados (ahora en formato string)
vector_bow = vectorizer_bow.fit_transform(df['Comentarios_sin_StopWords_str'])

# Convertimos la matriz de características en un DataFrame para visualizar
bow_df = pd.DataFrame(vector_bow.toarray(), columns=vectorizer_bow.get_feature_names_out())

# Obtener los nombres de las características (palabras)
features_bow = vectorizer_bow.get_feature_names_out()


# Crear un DataFrame con las frecuencias de las palabras
df_bow = pd.DataFrame(vector_bow.toarray(), columns=features_bow)

In [ ]:
# Ajustar y transformar los datos de entrenamiento
X_train_bow = vectorizer_bow.fit_transform(X_train)
# Transformar los datos de prueba
X_test_bow  = X_test_bow  = vectorizer_bow.transform(X_test)

Generación y prueba del Modelo con BoW.

In [ ]:
# Crear un modelo de regresión logística
# Abajo tenés un código con los parámetros expresados de forma que puedas ir modificandolos
model_log_reg_Bow = LogisticRegression() #Instanciamos el modelo

# Entrenar el modelo
model_log_reg_Bow.fit(X_train_bow, y_train) # Fiteamos, es decir, el modelo aprende a partir de los datos de entrenamiento

# Hacer predicciones en el conjunto de prueba
y_pred_log_reg_Bow = model_log_reg_Bow.predict(X_test_bow) # Predecir

#### Evaluación del Modelo.


Matriz de Confusión.

In [ ]:
# 1. Matriz de confusión

# La Matriz de Confusión es útil para Muestra los aciertos y errores del modelo organizados por clase.

cm1 = confusion_matrix(y_test, y_pred_log_reg_Bow)
labels = ['Negativo', 'Positivo']

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.title('Matriz de Confusiónc con BoW')
plt.show()

Curva ROC AUC.

In [ ]:
# 2. Curva ROC
# ROC AUC SCORE evalúa qué tan bien el modelo separa las clases.
fpr1, tpr1, _ = roc_curve(y_test, model_log_reg_Bow.decision_function(X_test_bow))
roc_auc1 = roc_auc_score(y_test, model_log_reg_Bow.decision_function(X_test_bow))

plt.figure(figsize=(8, 6))
plt.plot(fpr1, tpr1, color='darkorange', lw=2, label=f'Área bajo la curva ROC AUC = {roc_auc1:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

Métricas de Predicción.

In [ ]:
# 3. Métricas
# Accuracy:Para medir qué tan bien predice el modelo en datos nuevos (exactitud).
# Accuracy mide el porcentaje total de predicciones correctas sobre el total de casos.
accuracy = accuracy_score(y_test, y_pred_log_reg)
# Precision: Para medir el costo de un falso positivo es alto (por ejemplo, recomendar una película mala como buena).
# Precision mide qué proporción de las predicciones positivas hechas por el modelo son realmente positivas.
precision = precision_score(y_test, y_pred_log_reg)
# Recall: Para medir cuántos de los casos positivos reales fueron capturados por el modelo.
recall = recall_score(y_test, y_pred_log_reg)
# f1 Score: Para medir el promedio armónico entre precisión y recall. Un buen balance si ambas cosas son importantes.
f1 = f1_score(y_test, y_pred_log_reg)

print("Métricas de desempeño del modelo:")
print(f"Accuracy : {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall   : {recall:.2f}")
print(f"F1 Score : {f1:.2f}")

Validación Cruzada con BoW.

In [ ]:
# Pipeline con Bag of Words y Regresión Logística
pipeline1 = make_pipeline(
    CountVectorizer(max_features=5000),  # Bag of Words
    LogisticRegression()
)

# Validación cruzada con 5 particiones
scores = cross_val_score(pipeline1, df['Comentario'], df['Valor'], cv=5, scoring='accuracy')

# Resultados
print(f"Precisión media con validación cruzada (BoW): {scores.mean():.3f}")
print(f"Desviación estándar: {scores.std():.3f}")

## Redes Neuronales.

### Prueba de Modelo con Keras.

In [ ]:
# Vectorizar los textos
vectorizer_Neuro = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
X = vectorizer_Neuro.fit_transform(df['Comentarios_sin_StopWords_str']).toarray()

In [ ]:
# Variable objetivo
y = df['Valor'].values  # Asegurate que sea 0 y 1

In [ ]:
# Dividir en train y test (estratificado)
X_train1, X_val, y_train1, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train1.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # 1 neurona y activación sigmoid para binario

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',   # función para binaria
              metrics=['accuracy'])

In [ ]:
# Mostramos el resumen del modelo: esto nos dará detalles sobre cada capa y el número de parámetros entrenables en el modelo.
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Use X_train1 and y_train1 which were specifically prepared for the neural network
history = model.fit(X_train1, y_train1,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=10,
                    callbacks=[early_stop])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = (model.predict(X_val) > 0.5).astype("int32")

print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))

📊 Gráfico de Accuracy y Loss por Época

In [ ]:
# Accuracy
plt.figure(figsize=(12,5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento', color='#4caf50')
plt.plot(history.history['val_accuracy'], label='Validación', color='#2196f3')
plt.title('Precisión (Accuracy) por Época')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento', color='#f57c00')
plt.plot(history.history['val_loss'], label='Validación', color='#e53935')
plt.title('Pérdida (Loss) por Época')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_train1, y_train1, verbose=False)
print("Precisión Entrenamiento: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_val, y_val, verbose=False)
print("Precisión Prueba:  {:.4f}".format(accuracy))


Evaluación del modelo.

**El modelo tiene overfitting**

Si hay sobreajuste (overfitting), el modelo aprende demasiado bien los datos de entrenamiento, pero falla al generalizar en los datos nuevos (de validación o test).

📈 ¿Qué es accuracy?
Accuracy (precisión) mide el porcentaje de predicciones correctas que hace tu modelo.

Se suele graficar en función de las épocas (cada ciclo completo de entrenamiento con todos los datos).

✅ Interpretación:
Si la accuracy de entrenamiento y validación suben juntas, el modelo está aprendiendo bien.

Si la accuracy de entrenamiento sube pero la de validación se estanca o baja, puede estar sobreajustando (overfitting).

📉 ¿Qué es loss?
Loss es una medida del error que comete el modelo (por ejemplo, la entropía cruzada en clasificación).

El objetivo es minimizar la pérdida durante el entrenamiento.

✅ Interpretación:
Una loss que disminuye tanto en entrenamiento como validación = buen aprendizaje.

Si loss en validación sube mientras la de entrenamiento baja = probable sobreajuste.

In [ ]:
# Paso 1: Seleccionar una reseña real del DataFrame
indice = 1000  # Cambiá este número si querés ver otra reseña

oracion_real = df['Comentario'].iloc[indice]
valoracion_real = df['Valor'].iloc[indice]

# --- CAMBIO AQUÍ: Usar el mismo TF-IDF Vectorizer usado para entrenar la Red Neuronal ---
# Asumiendo que 'vectorizer_Neuro' es el TfidfVectorizer utilizado para crear X_train1 y X_val
# Es crucial que sea la *misma* instancia que fue fit_transformed en los datos de entrenamiento
# Si no es la misma instancia, o si X_train1/X_val fueron creados con otro vectorizador, ajusta esto.
# Basado en el código anterior, parece que vectorizer_Neuro fue usado justo antes de dividir X, y.
# Asegúrate de que la celda donde se define vectorizer_Neuro se ejecuta antes que esta.
nueva_reseña_vectorizada = vectorizer_Neuro.transform([oracion_real])

# Paso 4: Predecir con el modelo
# La predicción espera un tensor, y .transform() de TfidfVectorizer devuelve una matriz dispersa.
# Necesitamos convertirla a un array denso si el modelo Sequential fue construido para aceptar entradas densas
# como lo sugiere el error con Dense(input_shape=(X_train1.shape[1],)).
# Asumiendo que X_train1 fue .toarray() después de TF-IDF.
nueva_reseña_vectorizada_dense = nueva_reseña_vectorizada.toarray()

prediccion = model.predict(nueva_reseña_vectorizada_dense)

# Paso 5: Convertir la probabilidad a clase 0 o 1
valoracion_predicha = 1 if prediccion[0][0] >= 0.5 else 0

# Paso 6: Mostrar resultados
print(f"Reseña: {oracion_real}")
print(f"Valoración real: {valoracion_real}")
print(f"Valoración predicha: {valoracion_predicha}")
print(f"Probabilidad predicha: {prediccion[0][0]:.4f}")

In [ ]:
# Testeamos con nuevas oraciones

# Definir una nueva oración para predecir.
nueva_oracion = ["si es fatal"]

nueva_secuencia_vectorizada = vectorizer_Neuro.transform(nueva_oracion)

# Convertir a array denso si el modelo lo espera
nueva_secuencia_vectorizada_dense = nueva_secuencia_vectorizada.toarray()


# Usar el modelo para predecir la valoración (0 o 1)

prediccion = model.predict(nueva_secuencia_vectorizada_dense)

print(f"Predicción: {prediccion[0][0]}")
valoracion = 1 if prediccion[0][0] >= 0.5 else 0
print(f"Valoración predicha: {valoracion}")

In [ ]:
# X ahora contiene un array de secuencias numéricas (en formato tensor o matriz), en las que cada número representa un índice de palabra del vocabulario.
# Estas secuencias están ajustadas para tener la misma longitud (max_len=100), con las más largas recortadas y las más cortas rellenadas con ceros.
# Visualizamos el tipo de dato que es X
print(type(X))

print(X)

### 📦 Tuning con Keras Tuner para clasificación de sentimientos

Aprovechando X_train1, X_val, y_train1, y_val  utilizadas en la primer pureba con Keras, utilizamos las mismas variables paa buscar los mejores hiperparámetros.

#### 1. PADDEAR

In [ ]:
#Los modelos de tf.keras esperan que todas las secuencias de entrada tengan la misma longitud
#para poder agruparlas en una matriz (tensor) y procesarlas por lotes.

# Define the maximum length for padding sequences
max_len = 1000
X_train_pad = pad_sequences(X_train1, maxlen=max_len)
X_val_pad = pad_sequences(X_val, maxlen=max_len)

#### 🎯 2. Definir función para el modelo a tunear


In [ ]:
def build_model(hp):
    model = Sequential()

    model.add(Embedding(
        input_dim=max_words + 1,
        output_dim=hp.Choice("embedding_dim", [32, 64, 128]),
        input_length=max_len
    ))

    model.add(Bidirectional(SimpleRNN(
        units=hp.Int("rnn_units", min_value=16, max_value=64, step=16)
    )))

    model.add(Dropout(hp.Choice("dropout_rate", [0.2, 0.3, 0.5])))

    model.add(Dense(1, activation="sigmoid"))

    model.compile(
        optimizer=Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

#### 🔍 3. Inicializar Keras Tuner



In [ ]:
# Define el tamaño del vocabulario basado en el vectorizador TF-IDF
# utilizado anteriormente en la sección de Redes Neuronales.
# 'vectorizer_Neuro' fue fit_transform en 'df['Comentarios_sin_StopWords_str']'
max_words = len(vectorizer_Neuro.vocabulary_)

tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    directory="keras_tuner_dir",
    project_name="sentiment_rnn_tuning"
)

#### 🚀 4. Ejecutar búsqueda

In [ ]:
tuner.search(X_train_pad, y_train1, epochs=5, validation_data=(X_val_pad, y_val))

#### 🏆 5. Ver los mejores hiperparámetros

In [ ]:
best_hp = tuner.get_best_hyperparameters(1)[0]

print("🔧 Mejores hiperparámetros encontrados:")
for param in best_hp.values:
    print(f"{param}: {best_hp.get(param)}")

#### 📈 6. Reentrenar el mejor modelo con más épocas

In [ ]:
best_model = tuner.hypermodel.build(best_hp)

history = best_model.fit(
    X_train_pad, y_train1,
    validation_data=(X_val_pad, y_val),
    epochs=10,
    batch_size=32
)

#### 📊 Gráficos de entrenamiento vs validación

In [ ]:
# Extraer métricas del objeto history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(acc) + 1)

# Crear figura
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Entrenamiento')
plt.plot(epochs_range, val_acc, label='Validación')
plt.title('Precisión (accuracy)')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Entrenamiento')
plt.plot(epochs_range, val_loss, label='Validación')
plt.title('Pérdida (loss)')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# **Conclusiones.**

Regresion lineal utiliizando TF-IFD

🧠 Interpretación general:
Tu modelo tiene un rendimiento sólido y equilibrado, con buena capacidad tanto para detectar verdaderos positivos (recall) como para evitar errores en las predicciones positivas (precision). El F1 Score confirma este equilibrio.



# **Futuras Lineas.**